In [5]:
#!pip install anchorpy

In [4]:
import asyncio
import os
import requests
import json
import logging
from anchorpy import Provider, Wallet, Program
from solana.publickey import PublicKey
from solana.rpc.async_api import AsyncClient

RPC_Endpoint = 'https://api.mainnet-beta.solana.com'
program_id = "cjg3oHmg9uuPsP8D6g29NWvhySJkdYdAo9D25PRbKXJ"

async def get_state(address, program_id, provider_uris):
    """Fetch IDL file from the chain"""
    for index, provider_uri in enumerate(provider_uris):
        try:
            logging.info(f"Trying uri {provider_uri}")
            client = AsyncClient(provider_uri)
            provider = Provider(client, Wallet.dummy())
            program_id_key = PublicKey(program_id)
            
            program = await Program.at(program_id_key, provider)
            logging.debug(program.idl)
            
            address_state = await program.account['State'].fetch(address)
            logging.debug(address_state)
            
            await program.close()

            return address_state
        except Exception as e:
            if index < (len(provider_uris) - 1):
                logging.exception("An exception occurred. Trying another uri")
            else:
                await program.close()
                raise e

def getTokenAccountBalance(token_vault, provider_uri):
    data =   {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "getTokenAccountBalance",
        "params": [
          str(token_vault),
                {
                "commitment": "finalized",
                "encoding":"jsonParsed"
              } 
        ]
      }
    headers = {"Content-Type": "application/json"}

    ret = requests.post(provider_uri, data=json.dumps(data), headers=headers)
    ret = json.loads(ret.text)
    if 'result' in ret:
        ret = ret['result']['value']["amount"]
    else:
        ret = 0
    return ret

async def getLinkAvailableForPayment(address, program_id, provider_uri):
    try:
        state = await get_state(address, program_id, [provider_uri])
    except:
        state = None
        
    remaining = None     
    if state:
        countUnpaidRounds, reimbursements = 0, 0
        for oracle in state.oracles.xs:
            numRounds = int(state.config.latest_aggregator_round_id) - int(oracle.from_round_id)
            if numRounds < 0:
                numRounds = 0

            countUnpaidRounds += int(numRounds)
            reimbursements += oracle.payment_gjuels

        linkBalance = getTokenAccountBalance(state.config.token_vault, provider_uri)
        logging.debug(f'linkBalance {linkBalance}')
        if linkBalance:
            amountDue = (state.config.billing.observation_payment_gjuels)*countUnpaidRounds + reimbursements
            logging.debug(f'amountDue {amountDue}')
            remaining = int(linkBalance) - amountDue  
            
    return remaining


In [5]:
if __name__ == '__main__':
    logging.basicConfig(
                format='%(asctime)s %(funcName)s %(levelname)s %(message)s',
                level=os.environ.get('LOGLEVEL', 'DEBUG').upper(),
                datefmt='%Y-%m-%d %H:%M:a%S',
                )
    LinkAvailableForPayment = await getLinkAvailableForPayment(
                                    "6Ho7WLD9QiZLy25Te67wxTyP1C68FzumNa1ZuUKafYad",
                                     program_id,
                                     RPC_Endpoint, 
                                    )
    logging.info(f'LinkAvailableForPayment {LinkAvailableForPayment}')

2022-05-24 23:08:a02 get_state INFO Trying uri https://api.mainnet-beta.solana.com
2022-05-24 23:08:a02 _before_request DEBUG Making HTTP request. URI: https://api.mainnet-beta.solana.com, RequestID: 1, Method: getAccountInfo, Params: ('2WAC1fCfVFWjPYc9FdjzZtRMZrz9dnkXC4fnweQ79THf', {'encoding': 'base64', 'commitment': 'finalized'})
2022-05-24 23:08:a03 _send_single_request DEBUG HTTP Request: POST https://api.mainnet-beta.solana.com "HTTP/1.1 200 OK"
2022-05-24 23:08:a03 _after_request DEBUG Getting response HTTP. URI: https://api.mainnet-beta.solana.com, Method: getAccountInfo, Response: {"jsonrpc":"2.0","result":{"context":{"slot":135023417},"value":{"data":["GEZivzqQe57P6NX1pCLn/VzSe+aTFLcPVHEXx6ZgfFLI6Nwgn23vxnwGAAB4nO1aWW/jNhD+K4KejYXtpGmSN++JoMk6iNNt0cJY0BLlsJFIlaSceAP/946oi9St3Cn20eTMcOab4RyU7+wN5oIwah/bk3fjd2N7ZFMUYPjJHD6FXw6jQiIqhX389122dzb78/v8Yvbh9NMCSOQ2hMU728UeodiF/UiQH9jejewN8qOYYXIEv3L2jydfPi0uvy9O/vrUj31vau+WI5uAKjxyJOhrqEMokQT5MdPIRo7DopK+YIGM94g4i6R9DEJw/GNB1hRz+9hDv

2022-05-24 23:08:a03 _before_request DEBUG Making HTTP request. URI: https://api.mainnet-beta.solana.com, RequestID: 2, Method: getAccountInfo, Params: ('6Ho7WLD9QiZLy25Te67wxTyP1C68FzumNa1ZuUKafYad', {'encoding': 'base64', 'commitment': 'finalized'})
2022-05-24 23:08:a03 _send_single_request DEBUG HTTP Request: POST https://api.mainnet-beta.solana.com "HTTP/1.1 200 OK"
2022-05-24 23:08:a03 _after_request DEBUG Getting response HTTP. URI: https://api.mainnet-beta.solana.com, Method: getAccountInfo, Response: {"jsonrpc":"2.0","result":{"context":{"slot":135023417},"value":{"data":["2JJrXmhLtrEB/wAAAAAAAEiY7WUfIpZAj4kIO/FMvOIDIMnanR9Ry+yR52mGAWOAmcmtruxhki7kTql3Sq+L7LkV32xlWv/oKbdEyvxvZhIAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAI2sVon8VdK+ObintRqMb4gBBgi4CWtSoXVa1DDgfd40+s+ZU2fBQeHc9E5fQ0QiIRTe2Km1O+jXvSv9xHc40Ad9YfNM4QYHE//XaEyUpbVyUrPze4vUhFGncqOffHtbX6RLdr7HNddJGoWbtQHaX3eTCESEsDu6NmMCsISHXoulQEIPAAAAAAAAAAAAAAAAAADodkgXAAAAAAAAAAAAAAAFAwAARBsBADKdBABdQTIE2dYZs7XoGcpSA3PlArGKcu7k2eiB

2022-05-24 23:08:a03 get_state DEBUG State(version=1, vault_nonce=255, padding0=0, padding1=0, feed=5tPc2esGRQdzT1J7jFrsBMMGwm21hZLUbTMt7Qma4o27, config=Config(owner=BMKk78WEmZCQPptNYET8jdwf4EnsXY4WS74TBWjiSAbb, proposed_owner=11111111111111111111111111111111, token_mint=AY2uKQZ21PGHoarsdnJ65xwJv1ojnM1Vn8UGMRpfZ2nX, token_vault=Ht4XWR59DfUfSrFTqgcEXRWJFKD5J9sRHvvYcb9FdDbc, requester_access_controller=9SSbDuWx4kjb91bVABofejAdT8Ax6Pc8JNkkM4zjq8gE, billing_access_controller=C4LbydwHQj38raQgEoBjk6Ugv9mwrB6PuxDCCBinxqDe, min_answer=1000000, max_answer=100000000000, f=5, round=3, padding0=0, epoch=72516, latest_aggregator_round_id=302386, latest_transmitter=7H2aMNigJrHi5TtXtDtEN9NFQRp5x7GQR48SUWdZ7SnW, config_count=1, latest_config_digest=ListContainer([0, 3, 27, 171, 151, 197, 158, 6, 119, 12, 242, 95, 197, 36, 184, 224, 86, 10, 22, 86, 243, 185, 185, 124, 98, 102, 98, 16, 118, 7, 63, 196]), latest_config_block_number=121435701, billing=Billing(observation_payment_gjuels=12440, transmission

2022-05-24 23:08:a03 _new_conn DEBUG Starting new HTTPS connection (1): api.mainnet-beta.solana.com:443
2022-05-24 23:08:a04 _make_request DEBUG https://api.mainnet-beta.solana.com:443 "POST / HTTP/1.1" 200 None
2022-05-24 23:08:a04 getLinkAvailableForPayment DEBUG linkBalance 1100010001000
2022-05-24 23:08:a04 getLinkAvailableForPayment DEBUG amountDue 235676888353831000
2022-05-24 23:08:a04 <module> INFO LinkAvailableForPayment -235675788343830000
